In [ ]:
import pynput  # Library for capturing keyboard input
import time
from collections import defaultdict

# Data structure to hold keystroke timing information
keystrokes = defaultdict(list)
start_time = time.time()

def on_press(key):
    current_time = time.time()
    try:
        keystrokes[key.char].append((current_time - start_time, 'press'))
    except AttributeError:
        keystrokes[str(key)].append((current_time - start_time, 'press'))  # For special keys

def on_release(key):
    current_time = time.time()
    try:
        keystrokes[key.char].append((current_time - start_time, 'release'))
    except AttributeError:
        keystrokes[str(key)].append((current_time - start_time, 'release'))

    # Stop listener if 'esc' is pressed (for testing, remove in production)
    if key == pynput.keyboard.Key.esc:
        return False

# Start listener
listener = pynput.keyboard.Listener(on_press=on_press, on_release=on_release)
listener.start()
listener.join()

# Save data securely (implement encryption)
import json
from cryptography.fernet import Fernet

key = Fernet.generate_key()
cipher_suite = Fernet(key)
encrypted_data = cipher_suite.encrypt(json.dumps(keystrokes).encode())

# Save to file
with open('keystrokes_data.enc', 'wb') as file:
    file.write(encrypted_data)

In [ ]:
def extract_features(data):
    features = []
    for key, events in data.items():
        for i in range(0, len(events) - 1, 2):  # Step by 2 to pair press and release
            if i + 1 < len(events):
                press_time, _ = events[i]
                release_time, _ = events[i + 1]
                hold_time = release_time - press_time
                if i + 2 < len(events):
                    next_press_time, _ = events[i + 2]
                    flight_time = next_press_time - release_time
                    features.append([hold_time, flight_time])
    return features

features = extract_features(keystrokes)

In [ ]:
from sklearn.ensemble import IsolationForest
import numpy as np

# Convert list of lists to numpy array for sklearn
X = np.array(features)

# Train the model
model = IsolationForest(contamination=0.1, random_state=42)
model.fit(X)

# Save the model for later use
import joblib
joblib.dump(model, 'isolation_forest_model.joblib')

In [ ]:
# Load the model
loaded_model = joblib.load('isolation_forest_model.joblib')

# Simulate real-time input (in reality, this would be from live keystrokes)
new_keystrokes = extract_features(keystrokes)  # This would be real-time data
new_data = np.array(new_keystrokes)

# Predict if the new data points are anomalies
predictions = loaded_model.predict(new_data)

# Alert on anomalies
for i, pred in enumerate(predictions):
    if pred == -1:  # -1 indicates an anomaly in Isolation Forest
        print(f"Anomaly detected at index {i} with data {new_data[i]}")